In [2]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_openml

In [3]:
boston = fetch_openml(name="boston", version=1, as_frame=True)

In [4]:
X_boston = boston.data
y_boston = boston.target

In [5]:
print("Dataset features name: "+str(boston.feature_names))
print("Dataset features size: "+str(boston.data.shape))
print("Dataset target size: "+str(boston.target.shape))

Dataset features name: ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']
Dataset features size: (506, 13)
Dataset target size: (506,)


In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score

In [9]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test = train_test_split(X_boston,y_boston,train_size=0.80,test_size=0.20,random_state=123)
print("Train/Test Sets Sizes: ", X_train.shape,X_test.shape,Y_train.shape,Y_test.shape)

Train/Test Sets Sizes:  (404, 13) (102, 13) (404,) (102,)


In [10]:
lr = LinearRegression()
dt = DecisionTreeRegressor()
knn = KNeighborsRegressor()

In [11]:
lr.fit(X_train,Y_train)
dt.fit(X_train,Y_train)
knn.fit(X_train,Y_train)

KNeighborsRegressor()

In [13]:
X_test = X_test.astype(float)
y_pred1 = lr.predict(X_test)
y_pred2 = dt.predict(X_test)
y_pred3 = knn.predict(X_test)

In [15]:
print("R^2 score for LR", r2_score(Y_test,y_pred1))
print("R^2 score for DT", r2_score(Y_test,y_pred2))
print("R^2 score for KNN", r2_score(Y_test,y_pred3))

R^2 score for LR 0.6592466510354096
R^2 score for DT 0.4413006626061766
R^2 score for KNN 0.5475962186976784


In [16]:
from sklearn.ensemble import BaggingRegressor

bag_regressor = BaggingRegressor(random_state=1)
bag_regressor.fit(X_train,Y_train)

BaggingRegressor(random_state=1)

In [18]:
Y_preds = bag_regressor.predict(X_test)

print("Training Coefficient of R^2: %.3f"%bag_regressor.score(X_train,Y_train))
print("Test Coefficient of R^2: %.3f"%bag_regressor.score(X_test,Y_test))

Training Coefficient of R^2: 0.980
Test Coefficient of R^2: 0.818


In [20]:
%%time

n_samples = boston.data.shape[0]
n_features = boston.data.shape[1]

params = {'estimator': [None, LinearRegression(), KNeighborsRegressor()],
          'n_estimators': [20,50,100],
          'max_samples': [0.5,1.0],
          'max_features': [0.5,1.0],
          'bootstrap': [True, False],
          'bootstrap_features': [True, False]}

bagging_regressor_grid = GridSearchCV(BaggingRegressor(random_state=1, n_jobs=-1), param_grid =params, cv=3, n_jobs=-1, verbose=1)
bagging_regressor_grid.fit(X_train, Y_train)

print('Train R^2 Score : %.3f'%bagging_regressor_grid.best_estimator_.score(X_train, Y_train))
print('Test R^2 Score : %.3f'%bagging_regressor_grid.best_estimator_.score(X_test, Y_test))
print('Best R^2 Score Through Grid Search : %.3f'%bagging_regressor_grid.best_score_)
print('Best Parameters : ',bagging_regressor_grid.best_params_)

Fitting 3 folds for each of 144 candidates, totalling 432 fits
Train R^2 Score : 0.983
Test R^2 Score : 0.805
Best R^2 Score Through Grid Search : 0.871
Best Parameters :  {'bootstrap': True, 'bootstrap_features': False, 'estimator': None, 'max_features': 1.0, 'max_samples': 1.0, 'n_estimators': 50}
CPU times: user 1.13 s, sys: 88.2 ms, total: 1.22 s
Wall time: 1min 11s
